In [405]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.svm import SVR

In [406]:
df = pd.read_csv('train_df.csv')

In [407]:
df.head()

property_type      sector  price  bedRoom  bathroom balcony   agePossession  \
0          flat  sohna road   0.73      2.0       2.0       3  Relatively New   
1          flat   sector 90   1.29      3.0       3.0       3  Relatively New   
2          flat  sector 107   0.75      2.0       2.0      3+  Relatively New   
3          flat  sector 102   2.50      4.0       5.0      3+  Relatively New   
4          flat  sector 67a   0.60      2.0       2.0       2    New Property   

   built_up_area  servant room  pooja room  study room  others  store room  \
0         1148.0           0.0         1.0         0.0     0.0         0.0   
1         1540.0           0.0         0.0         1.0     0.0         0.0   
2         1201.0           0.0         1.0         0.0     0.0         0.0   
3         2600.0           1.0         1.0         0.0     0.0         0.0   
4          633.0           0.0         0.0         0.0     0.0         0.0   

   furnishing_type luxury_category floor_category  
0              0.0          medium      Mid Floor  
1              0.0            high      Mid Floor  
2              0.0            high     High Floor  
3              1.0            high      Mid Floor  
4              0.0             low     High Floor

In [408]:
df.shape

(3554, 16)

In [409]:
df.drop(columns=['others'],axis=1,inplace=True)

In [410]:
df['sector'].value_counts()

sohna road    152
sector 85     108
sector 102    107
sector 92     100
sector 69      93
             ... 
sector 88b      3
sector 73       3
sector 27       2
sector 17a      1
sector 37       1
Name: sector, Length: 113, dtype: int64

In [411]:
df = df[~(df['sector']=='sector 27') & ~(df['sector']=='sector 17a') & ~(df['sector']=='sector 37')]

In [412]:
df.head()

property_type      sector  price  bedRoom  bathroom balcony   agePossession  \
0          flat  sohna road   0.73      2.0       2.0       3  Relatively New   
1          flat   sector 90   1.29      3.0       3.0       3  Relatively New   
2          flat  sector 107   0.75      2.0       2.0      3+  Relatively New   
3          flat  sector 102   2.50      4.0       5.0      3+  Relatively New   
4          flat  sector 67a   0.60      2.0       2.0       2    New Property   

   built_up_area  servant room  pooja room  study room  store room  \
0         1148.0           0.0         1.0         0.0         0.0   
1         1540.0           0.0         0.0         1.0         0.0   
2         1201.0           0.0         1.0         0.0         0.0   
3         2600.0           1.0         1.0         0.0         0.0   
4          633.0           0.0         0.0         0.0         0.0   

   furnishing_type luxury_category floor_category  
0              0.0          medium      Mid Floor  
1              0.0            high      Mid Floor  
2              0.0            high     High Floor  
3              1.0            high      Mid Floor  
4              0.0             low     High Floor

In [413]:
df.shape

(3550, 15)

In [414]:
df['furnishing_type'].value_counts()

0.0    2347
1.0    1016
2.0     187
Name: furnishing_type, dtype: int64

In [415]:
df.loc[df['furnishing_type']==0.0,'furnishing_type'] = 'unfurnished'
df.loc[df['furnishing_type']==1.0,'furnishing_type'] = 'semifurnished'
df.loc[df['furnishing_type']==2.0,'furnishing_type'] = 'furnished'

In [416]:
df['furnishing_type'].value_counts()

unfurnished      2347
semifurnished    1016
furnished         187
Name: furnishing_type, dtype: int64

In [417]:
df.head()

property_type      sector  price  bedRoom  bathroom balcony   agePossession  \
0          flat  sohna road   0.73      2.0       2.0       3  Relatively New   
1          flat   sector 90   1.29      3.0       3.0       3  Relatively New   
2          flat  sector 107   0.75      2.0       2.0      3+  Relatively New   
3          flat  sector 102   2.50      4.0       5.0      3+  Relatively New   
4          flat  sector 67a   0.60      2.0       2.0       2    New Property   

   built_up_area  servant room  pooja room  study room  store room  \
0         1148.0           0.0         1.0         0.0         0.0   
1         1540.0           0.0         0.0         1.0         0.0   
2         1201.0           0.0         1.0         0.0         0.0   
3         2600.0           1.0         1.0         0.0         0.0   
4          633.0           0.0         0.0         0.0         0.0   

  furnishing_type luxury_category floor_category  
0     unfurnished          medium      Mid Floor  
1     unfurnished            high      Mid Floor  
2     unfurnished            high     High Floor  
3   semifurnished            high      Mid Floor  
4     unfurnished             low     High Floor

In [418]:
X = df.drop(columns='price')
y = df['price']

In [419]:
X['sector'].value_counts()

sohna road    152
sector 85     108
sector 102    107
sector 92     100
sector 69      93
             ... 
sector 40       5
sector 30       5
sector 88b      3
sector 17b      3
sector 73       3
Name: sector, Length: 110, dtype: int64

In [420]:
print(X.shape)
print(y.shape)

(3550, 14)
(3550,)


In [421]:
y_transformed = np.log1p(y)

In [422]:
X.columns

Index(['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'pooja room',
       'study room', 'store room', 'furnishing_type', 'luxury_category',
       'floor_category'],
      dtype='object')

In [423]:
col_encode = ['property_type', 'sector', 'balcony',
       'agePossession', 'furnishing_type', 'luxury_category',
       'floor_category']

In [424]:
preprocessor = ColumnTransformer(
transformers = [
    ('num',StandardScaler(),['bedRoom','bathroom','built_up_area','servant room', 'pooja room','study room', 'store room']),
    ('cat',OrdinalEncoder(),col_encode)
],remainder='passthrough')

In [425]:
pipeline = Pipeline([
    ('preprocessor',preprocessor),
    ('regressor',LinearRegression())
])

In [426]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [427]:
scores.mean()

0.7401577314948896

In [428]:
X_train,X_test,y_train,y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=45)

In [429]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room', 'pooja room',
                                                   'study room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', L

In [430]:
y_pred = pipeline.predict(X_test)

In [431]:
y_pred = np.expm1(y_pred)

In [432]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.8329424452355138

In [433]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=40)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [434]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [435]:
model_score = []
for model_name,model in model_dict.items():
    model_score.append(scorer(model_name,model))

In [436]:
model_score

[['linear_reg', 0.7401577314948896, 0.8788939830629509],
 ['svr', 0.7644359231024276, 0.8235906162429296],
 ['ridge', 0.7401627681817147, 0.878859445799515],
 ['LASSO', 0.056920648442563226, 1.5221836668612472],
 ['decision tree', 0.787881298973606, 0.6429114828350291],
 ['random forest', 0.883955871947933, 0.5022827332765376],
 ['extra trees', 0.8669443212522138, 0.5739079334336339],
 ['gradient boosting', 0.8773899528169984, 0.5361854130646923],
 ['adaboost', 0.7593177001463017, 0.8212555313695499],
 ['mlp', 0.8075139577212133, 0.8234913571612805],
 ['xgboost', 0.8924087796748988, 0.4677748155795353]]

In [437]:
model_score = pd.DataFrame(model_score,columns = ['Model','Score','MAE'])

In [438]:
model_score.sort_values('MAE',ascending=True)

Model     Score       MAE
10            xgboost  0.892409  0.467775
5       random forest  0.883956  0.502283
7   gradient boosting  0.877390  0.536185
6         extra trees  0.866944  0.573908
4       decision tree  0.787881  0.642911
8            adaboost  0.759318  0.821256
9                 mlp  0.807514  0.823491
1                 svr  0.764436  0.823591
2               ridge  0.740163  0.878859
0          linear_reg  0.740158  0.878894
3               LASSO  0.056921  1.522184

### OneHotEncoding

In [439]:
df.head()

property_type      sector  price  bedRoom  bathroom balcony   agePossession  \
0          flat  sohna road   0.73      2.0       2.0       3  Relatively New   
1          flat   sector 90   1.29      3.0       3.0       3  Relatively New   
2          flat  sector 107   0.75      2.0       2.0      3+  Relatively New   
3          flat  sector 102   2.50      4.0       5.0      3+  Relatively New   
4          flat  sector 67a   0.60      2.0       2.0       2    New Property   

   built_up_area  servant room  pooja room  study room  store room  \
0         1148.0           0.0         1.0         0.0         0.0   
1         1540.0           0.0         0.0         1.0         0.0   
2         1201.0           0.0         1.0         0.0         0.0   
3         2600.0           1.0         1.0         0.0         0.0   
4          633.0           0.0         0.0         0.0         0.0   

  furnishing_type luxury_category floor_category  
0     unfurnished          medium      Mid Floor  
1     unfurnished            high      Mid Floor  
2     unfurnished            high     High Floor  
3   semifurnished            high      Mid Floor  
4     unfurnished             low     High Floor

In [440]:
col_encode1 = ['property_type', 'balcony', 'luxury_category', 'floor_category']

In [441]:
preprocessor = ColumnTransformer(
transformers = [
    ('num',StandardScaler(),['bedRoom','bathroom','built_up_area','servant room', 'pooja room','study room', 'store room']),
    ('cat',OrdinalEncoder(),col_encode1),
    ('cat1',OneHotEncoder(),['sector','agePossession','furnishing_type'])
],remainder='passthrough')

In [442]:
pipeline = Pipeline([
    ('preprocessor',preprocessor),
    ('regressor',LinearRegression())
])

In [443]:
kfold = KFold(n_splits=10, shuffle=True, random_state=49)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [444]:
scores.mean()

0.8590930257961459

In [445]:
scores.std()

0.017045219318030498

In [446]:
X_train,X_test,y_train,y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=49)

In [447]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room', 'pooja room',
                                                   'study room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'balcony',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1', OneHotEncoder(),
                                                  ['sector', 'agePossession',
                             

In [448]:
y_pred = pipeline.predict(X_test)

In [449]:
y_pred = np.expm1(y_pred)

In [450]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.5888283377483179

In [451]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=40)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [452]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [453]:
model_score = []
for model_name,model in model_dict.items():
    model_score.append(scorer(model_name,model))

In [454]:
model_score

[['linear_reg', 0.8574218298814635, 0.5876948506897935],
 ['svr', 0.8779980149769401, 0.5170745419031955],
 ['ridge', 0.8575758255260052, 0.5936339401192701],
 ['LASSO', -0.003285432506095298, 1.562711475679067],
 ['decision tree', 0.7666299595667978, 0.6668780641524439],
 ['random forest', 0.8755433776080619, 0.5103052001189383],
 ['extra trees', 0.8887574110893117, 0.468754526541768],
 ['gradient boosting', 0.8603699592563172, 0.5654447996906731],
 ['adaboost', 0.7276513516152261, 0.8957152838475602],
 ['mlp', 0.8765435154026795, 0.5287544258405957],
 ['xgboost', 0.8868942041162363, 0.4681930743707736]]

In [455]:
model_df = pd.DataFrame(model_score, columns=['name','r2','mae'])

In [456]:
model_df.sort_values('mae',ascending=False)

name        r2       mae
3               LASSO -0.003285  1.562711
8            adaboost  0.727651  0.895715
4       decision tree  0.766630  0.666878
2               ridge  0.857576  0.593634
0          linear_reg  0.857422  0.587695
7   gradient boosting  0.860370  0.565445
9                 mlp  0.876544  0.528754
1                 svr  0.877998  0.517075
5       random forest  0.875543  0.510305
6         extra trees  0.888757  0.468755
10            xgboost  0.886894  0.468193

In [457]:
df.head()

property_type      sector  price  bedRoom  bathroom balcony   agePossession  \
0          flat  sohna road   0.73      2.0       2.0       3  Relatively New   
1          flat   sector 90   1.29      3.0       3.0       3  Relatively New   
2          flat  sector 107   0.75      2.0       2.0      3+  Relatively New   
3          flat  sector 102   2.50      4.0       5.0      3+  Relatively New   
4          flat  sector 67a   0.60      2.0       2.0       2    New Property   

   built_up_area  servant room  pooja room  study room  store room  \
0         1148.0           0.0         1.0         0.0         0.0   
1         1540.0           0.0         0.0         1.0         0.0   
2         1201.0           0.0         1.0         0.0         0.0   
3         2600.0           1.0         1.0         0.0         0.0   
4          633.0           0.0         0.0         0.0         0.0   

  furnishing_type luxury_category floor_category  
0     unfurnished          medium      Mid Floor  
1     unfurnished            high      Mid Floor  
2     unfurnished            high     High Floor  
3   semifurnished            high      Mid Floor  
4     unfurnished             low     High Floor

### Target Encoder

In [458]:
import category_encoders as ce

col_encode2 = ['property_type', 'balcony', 'luxury_category', 'floor_category']

preprocessor = ColumnTransformer(
transformers = [
    ('num',StandardScaler(),['bedRoom','bathroom','built_up_area','servant room', 'pooja room','study room', 'store room']),
    ('cat',OrdinalEncoder(),col_encode2),
    ('cat1',OneHotEncoder(drop='first'),['agePossession','furnishing_type']),
    ('target',ce.TargetEncoder(),['sector'])
],remainder='passthrough')

In [459]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=40)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [460]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [461]:
model_score = []
for model_name,model in model_dict.items():
    model_score.append(scorer(model_name,model))

In [462]:
model_score

[['linear_reg', 0.8301887532161663, 0.666150117461192],
 ['svr', 0.8594897011177887, 0.5767193887932812],
 ['ridge', 0.8302092057118445, 0.6665070812065587],
 ['LASSO', -0.003285432506095298, 1.562711475679067],
 ['decision tree', 0.8140003264183608, 0.6082878691133584],
 ['random forest', 0.8976025543017585, 0.4485811089273449],
 ['extra trees', 0.8941919968793546, 0.4590099627450659],
 ['gradient boosting', 0.8856269128994967, 0.5033774399967107],
 ['adaboost', 0.8181679586906115, 0.70998553919911],
 ['mlp', 0.8509466223734137, 0.5862286047492605],
 ['xgboost', 0.9002771811068164, 0.46789594021145725]]

In [463]:
model_df = pd.DataFrame(model_score, columns=['name','r2','mae'])

In [464]:
model_df.sort_values('mae',ascending=True)

name        r2       mae
5       random forest  0.897603  0.448581
6         extra trees  0.894192  0.459010
10            xgboost  0.900277  0.467896
7   gradient boosting  0.885627  0.503377
1                 svr  0.859490  0.576719
9                 mlp  0.850947  0.586229
4       decision tree  0.814000  0.608288
0          linear_reg  0.830189  0.666150
2               ridge  0.830209  0.666507
8            adaboost  0.818168  0.709986
3               LASSO -0.003285  1.562711

### Exporting Model

In [465]:
col_encode1 = ['property_type', 'balcony', 'luxury_category', 'floor_category']

In [466]:
preprocessor = ColumnTransformer(
transformers = [
    ('num',StandardScaler(),['bedRoom','bathroom','built_up_area','servant room', 'pooja room','study room', 'store room']),
    ('cat',OrdinalEncoder(),col_encode1),
    ('cat1',OneHotEncoder(),['sector','agePossession','furnishing_type'])
],remainder='passthrough')

In [467]:
pipeline = Pipeline([
    ('preprocessor',preprocessor),
    ('regressor',LinearRegression())
])

In [468]:
pipeline.fit(X,y_transformed)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room', 'pooja room',
                                                   'study room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'balcony',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1', OneHotEncoder(),
                                                  ['sector', 'agePossession',
                             

In [469]:
import pickle

with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [470]:
with open('df.pkl', 'wb') as file:
    pickle.dump(X, file)